In [ ]:
# IS NOT USED DUE TO NAs IN THE RESULTS!

from econml.iv.nnet import DeepIV

import keras
import numpy as np


import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from joblib import Parallel, delayed

In [ ]:
# Load dataset
dataset_rpe1 = pd.read_csv("data/sec6.2/data/dataset_rpe1.csv")

# Select relevant columns
interv_genes = dataset_rpe1.columns[:9].tolist()
train_data = dataset_rpe1[dataset_rpe1['interventions'].isin(interv_genes + ["non-targeting"])].copy()

# Convert intervention column to categorical
train_data['interventions'] = train_data['interventions'].astype('category')
train_data['Ztr'] = train_data.iloc[:, 10].astype('category')

# Get list of unique interventions (excluding "non-targeting")
unique_interventions = [g for g in train_data['interventions'].unique() if g != "non-targeting"]

# Define test data (from 50 test environments)
test_data_path = 'data/sec6.2/test_single_cell.csv'
test_data = pd.read_csv(test_data_path)
Xtest = test_data.iloc[:, 0:9].values  # Extract first 9 columns

In [ ]:
# Define treatment model (unchanged)
def build_treatment_model():
    return keras.Sequential([
        keras.layers.Dense(256, activation='relu', input_shape=(10,)),
        keras.layers.Dropout(0.17),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.17),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dropout(0.17),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dropout(0.17)
    ])

# Define response model (unchanged)
def build_response_model():
    return keras.Sequential([
        keras.layers.Dense(256, activation='relu', input_shape=(10,)),
        keras.layers.Dropout(0.17),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.17),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dropout(0.17),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dropout(0.17),
        keras.layers.Dense(1)
    ])

keras_fit_options = { "epochs": 30,
                      "validation_split": 0.2,
                      "callbacks": [keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)]}

# Define function to train DeepIV with one intervention removed
def train_deepIV_pairwise(i, gene_to_remove):
    print(f"Iteration {i+1}: Removing intervention {gene_to_remove}")

    # Remove data for the given intervention
    valid_rows = train_data['interventions'].isin(["non-targeting"] + list(set(interv_genes) - {gene_to_remove}))
    filtered_data = train_data[valid_rows]

    # Convert `Ztr` to One-Hot Encoding AFTER removing one environment
    encoder = OneHotEncoder(sparse_output=False)
    Ztr_encoded = encoder.fit_transform(filtered_data[['Ztr']])

    # Extract features and target
    Xtr = filtered_data.iloc[:, :9].values  # First 9 columns
    Ytr = filtered_data.iloc[:, 9].values   # 10th column

    z = Ztr_encoded  # Encoded intervention variable
    t = Xtr          # First 9 columns (gene expressions)
    y = Ytr          # 10th column (response variable)
    x = np.zeros(Xtr.shape[0])  # Dummy placeholder if needed for consistency

    # Instantiate models (exact same architecture)
    treatment_model = build_treatment_model()
    response_model = build_response_model()

    # Define DeepIV estimator
    deepIvEst = DeepIV(
        n_components=10,  # Gaussian components in MDN
        m = lambda z, x : treatment_model(keras.layers.concatenate([z,x])), # treatment model
        h = lambda t, x : response_model(keras.layers.concatenate([t,x])),  # response model
        n_samples=10,  # More samples for better response estimation
        use_upper_bound_loss=False,  # Approximation to true loss
        n_gradient_samples=5,  # More gradient samples to reduce variance
        optimizer=keras.optimizers.Adam(learning_rate=0.0001), # Keras optimizer to use for training - see https://keras.io/optimizers/ 
        first_stage_options=keras_fit_options,
        second_stage_options=keras_fit_options
    )

    # Train DeepIV
    deepIvEst.fit(Y=y,T=t,X=x,Z=z)
    
    # Make predictions on test data
    y_hat_deepIV = deepIvEst.predict(Xtest, np.zeros(Xtest.shape[0]))

    return i, y_hat_deepIV  # Return index and predictions

In [ ]:
# Run the 9 iterations in parallel
if __name__ == "__main__":
    num_iterations = len(unique_interventions)

    results = Parallel(n_jobs=9)(
        delayed(train_deepIV_pairwise)(i, gene_to_remove)
        for i, gene_to_remove in enumerate(unique_interventions)
    )

    # Store predictions in DataFrame
    df_results = pd.DataFrame({f'Run_{i+1}': y_hat_deepIV.flatten() for i, y_hat_deepIV in results})

    df_results.columns = unique_interventions

    # Save results
    df_results.to_csv("results/sec6.2/deepiv_singlecell_pairwise.csv", index=False)